# An Optical Flow-based Symbol Recognition Method

<span style="color: red;">TODO: Megnézni más címeket is példának!</span>

Journal: http://scientificbulletin.upm.ro/?pag=submit

**Keywords**: Human Computer Interaction, Augmented Reality

**Abstract**: Symbol recognizer instead of a full OCR.

## Introduction

The research of the proper human-computer interaction is one of the fundamental problems of computer science.
The motivation behind them is to find the most natural way of the communication with the help of machines.
The increasing number of Augmented Reality and Virtual Reality applications signs the relevance and actuality of the presented topic.

The goal of this paper is to provide an alternative method for solving the symbol recognition problem where the input is a video stream.
In the considered scenario the user draws various symbols into the air in front of the camera.
There is a set of predefined symbols.
We have to provide an image processing method and a classification algorithm which can guess the symbol with sufficient accuracy.

The proposed symbol recognition method uses motion flow information for estimating the recently drawn symbols.
Compared to the classical object tracking methods it has the following advantages and disadvantages.

The main benefit of this approach is the shape and texture invariance.
Most of the available methods combines shape detection and object tracking techniques [].
Therefore, the reliability of the symbol recognition depends on the accuracy of these two different steps.

An object detection and a pose estimation method can provide detailed information about the tracked object.
In our use case these informations are ignored.
It means that, our system is unable to differentiate between a human hand and an arbitrary pointer device on the images.

[DTW-s cikk]

## Video stream processing method

The processing of the video stream is a multistep process.
The video stream is the sequence of coloured images.
We call these images as frames.
The format and the resolution of all frames are the same.
The elapsed time between two sequential frames can be considered as a constant value, denoted by $\Delta t$.
Let see the overview of the proposed method in Figure [].
We describe the details of the processing steps in the following subsections.

<span style="color: red;">TODO: Összerakni egy áttekintő ábrát a feldolgozási folyamatról!</span>

<img src="./docs/images/processing-flow/processing-flow.png" width="80%">

### Image proprocessing

We consider only that cases, where the hue of objects on the images do not provide significant additional information in the aspect of symbol recognition.
Our experiment suggest that the usage of grayscale images is sufficient.
There are different methods for the grayscale convertion.
The selection of them does not play key role in the further processing steps.

We would like to reduce the effect of light condition changes.
Some of the video capturing devices have capability to adapt to them.
It helps to improve the image quality in the typical use case, but also means that any local intensity changes affect intensity changes globally.
Fortunately, the estimation of motion vectors will eliminate this kind of noise automatically.

### Motion vector field estimation

On sequential frames we can observe that some regions of the image are changing their positions.
It does not necessarily match with the real position changes of the objects of the captured scene.
Theoretically, it is possible to estimate the motion vector for any point of a frame.
Therefore, it results a vector field on the domain of the image.

The presented symbol recognition method based on the estimation of these motion vectors.
The following paragraphs mention the reasons which makes the estimation task difficult.

The *motion* is not a well-defined term in this consideration.
It is an intuitive concept without a strict mathematical foundation.

The estimation of a motion vector at a point requires the current frame and at least the previous one.
On the analogy of interpolation methods, we could take into account more preceding frames.
Expectedly, more frames are unable to cause significantly better classification accuracy.

The frames are rasterized images.
Smaller number of motion vectors is enough.

We have to select the tracked parts of the image.
The following properties are preferred.
* The union of these parts should result the original image.
* All parts should have exactly the same shape and size.
The segmentation of the image by a squared grid fulfils these requirements.
Moreover, it provides a more convenient calculation scheme.

The OpenCV library contains the implementation of an optical flow method of Lucas-Kanade [].
We use an equidistant squared grid on the image.
* The grid size is a free parameter of the method.
* The window size is $50 \times 50$.
* We have choosed the maximal level of the image pyramid to 2.
* The iterative search process termination criteria is $\varepsilon = 0.03$ and the maximal count of iteration steps is 10.

The implementation details of the algorithm and the tuning of proper parametrization is out of the scope of our current research.

<span style="color: red;">TODO: Készíteni egy ábrát a rácsról!</span>

<img src="./docs/images/grid/grid.png" width="50%">

<span style="color: red;">TODO: Gyári implementációt említeni!</span>

### Heat map calculation

The estimated motion vectors describe the drawed symbol in time intervals with $\Delta t$ lengths.
For higher level view we have to aggregate the vector fields.
A straightforward method of the aggregation is the usage of matrix which resembles to a heat map.

Let consider only the measure of the motions which is available as the lengths of motion vectors.
Let denote $l_i \in \mathbb{R}$ the length of a motion vector of the $i$-th frame.
Let denote $h_i \in \mathbb{R}$ the *heat value* on the $i$-th frame.

$$
l_i, l_{i-1}, l_{i-2}, \ldots, l_{i-k}
$$
We can use a $k+1$ sized convolution kernel.
$C: \mathbb{R}^{k+1} \rightarrow \mathbb{R}$

We would like to find a proper aggregator function $f: \mathbb{R}^2 \rightarrow \mathbb{R}$ as
$$
h_i = f(h_{i-1}, l_i).
$$

It requires a time window.

<span style="color: red;">TODO: Készíteni képet egy hőtérképről!</span>

<img src="./docs/images/heatmap/heatmap.png" width="50%">

### Temporal filter for Symbol Canvas

The length of the drawed symbols can be differ significantly.
The following approaches can be applied for solving this issue.

* Usage of lower level primitives. (Lines, curves)
* Define a temporal filter which is able to detect the begin and end of the symbol drawing process.

We prefer to use the latter, because on successful time segment detection the symbol recognition task becomes the classical pattern recognition problem.

3 seconds limit ..when the average motion is under a specified limit.

### Symbol recognizer

We define the ideal patterns in the resolution of the grid.
These are the reference patterns.
We calculate the sum of squared distances of pixel intensities.
We select the reference image where this sum is minimal.

As an improved solution we could use ANN.

## Results

### Symbol selection

In this section we overview the different kind of noises.
Light equalization

movement noise reduction

## Discussion

## Conclusions

## Acknowledgement

<span style="color: red;">TODO: Rakaczki Csabát megkérdezni, hogy kell-e bele ilyesmi!</span>

## References

20 items